In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import calendar


# Reading in Datasets

In [6]:
# Read the data into a Pandas DataFrame
ardd_fatal_crashes_df = pd.read_excel('ardd_fatal_crashes.xlsx')
ardd_fatal_crashes_df.head()


FileNotFoundError: [Errno 2] No such file or directory: 'ardd_fatal_crashes.xlsx'

In [ ]:
# Read the data into a Pandas DataFrame
ardd_fatalities_df = pd.read_excel('ardd_fatalities.xlsx')
ardd_fatalities_df.head()


In [ ]:
# Get a brief summary of the ardd_fatal_crashes DataFrame.
ardd_fatal_crashes_df.info()

In [ ]:
# Get a brief summary of the ardd_fatalities DataFrame.
ardd_fatalities_df.info()

In [ ]:
# Get the ardd_fatal_crashes_df columns.
ardd_fatal_crashes_df.columns

In [ ]:
# Get the ardd_fatalities_df columns.
ardd_fatalities_df.columns

In [ ]:
#Given Similarity in both datasets, drop common columns 
ardd_fatalities_df = ardd_fatalities_df.drop(['State', 'Month', 'Year', 'Dayweek', 'Time', 'Crash Type', 'Bus Involvement', 'Heavy Rigid Truck Involvement',
       'Articulated Truck Involvement', 'Speed Limit', 'National Remoteness Areas', 'SA4 Name 2021',
       'National LGA Name 2021', 'National Road Type', 'Christmas Period',
       'Easter Period', 'Day of week', 'Time of day'], axis=1)


In [ ]:
# Get the ardd_fatalities_df columns.
ardd_fatalities_df.columns

# Merging Datasets

In [ ]:
# Merge the dataframes
merged_df = pd.merge(ardd_fatal_crashes_df, ardd_fatalities_df, on='Crash ID', how='inner')


In [ ]:
# Get the merged_df columns.
merged_df.columns

In [ ]:
# Get a brief summary of the merged DataFrame.
merged_df.info()

In [ ]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 24)
merged_df

# Data Cleaning - Removing unique values.

In [ ]:
# Create an empty dictionary to store unique values for each column
unique_values = {}

# Loop through the columns and find unique values
for column in merged_df.columns:
    unique_values[column] = merged_df[column].unique()

# Display unique values for each column
for column, values in unique_values.items():
    print(f'Column: {column}')
    print(values)
    print('\n')
    

In [ ]:
# Define the list of special values you want to count
special_values = ['-9', 'nan', '<40', 'Unspecified', 'Unknown', 'Undetermined', 'Other/-9', 'U', 'M ']

# Create an empty dictionary to store the counts
special_value_counts = {}

# Iterate through the columns and count the special values
for column in merged_df.columns:
    column_counts = {}
    if merged_df[column].dtype == 'object':
        for special_value in special_values:
            if special_value == 'nan':
                count = merged_df[column].isna().sum()
            elif merged_df[column].dtype == 'str':
                count = (merged_df[column].str.strip() == special_value).sum()
            else:
                count = (merged_df[column] == special_value).sum()
            column_counts[special_value] = count
        special_value_counts[column] = column_counts

# Display the counts
for column, counts in special_value_counts.items():
    print(f'Column: {column}')
    for value, count in counts.items():
        print(f'{value}: {count}')
    print()


In [ ]:
# Define the list of special values you want to check for
special_values = ['-9', 'nan', '<40', 'Unspecified', 'Unknown', 'Undetermined', 'Other/-9', 'U', 'M ']

# Create a boolean mask to identify rows containing special values
special_value_mask = merged_df.isin(special_values)

# Count the rows with special values in each column
rows_with_special_values = special_value_mask.any(axis=1)

# Print the total number of rows with special values
total_rows_with_special_values = rows_with_special_values.sum()
print(f'Total rows with special values: {total_rows_with_special_values}')


In [ ]:
# Define the list of special values you want to remove
special_values = ['-9', 'nan', '<40', 'Unspecified', 'Unknown', 'Undetermined', 'Other/-9', 'U', 'M ']

# Create a boolean mask to identify rows containing special values
special_value_mask = merged_df.isin(special_values)

# Remove rows with special values from the DataFrame
filtered_df = merged_df[~special_value_mask.any(axis=1)]

# Print the shape of the filtered DataFrame
print(f'Shape of the filtered DataFrame: {filtered_df.shape}')


In [ ]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 24)
filtered_df

In [ ]:
# Save the filtered DataFrame as a CSV file
filtered_df.to_csv('filtered_data.csv', index=False)


# Data export to csv
## One. filtered_data is merged dataset with rows containing unique values dropped. 
### Rows containing '-9', 'NaN', '"BLANK-EMPTY CELL"' not removed. 

# Data Processing. 
## Filter data to last ten years 2013 - 2023

In [ ]:
# Load the 'filtered_df' from the CSV file
filtered_df = pd.read_csv('filtered_data.csv')

# Convert the 'Year' column to integers
filtered_df['Year'] = filtered_df['Year'].astype(int)

# Create a new DataFrame with only years between 2013 and 2023
filtered_df_2013_to_2023 = filtered_df[(filtered_df['Year'] >= 2013) & (filtered_df['Year'] <= 2023)]

# Reset the index if needed
filtered_df_2013_to_2023.reset_index(drop=True, inplace=True)

# Display the new DataFrame with years 2013 to 2023
filtered_df_2013_to_2023


In [ ]:
# Save the filtered DataFrame to a new CSV file
filtered_df_2013_to_2023.to_csv('filtered_data_with_years_2013_to_2023.csv', index=False)


## Create dataframes for export to csv to be used as database tables

In [ ]:
# ['Crash ID', 'State', 'Month', 'Year', 'Dayweek', 'Time', 'Crash Type',
#        'Bus Involvement', 'Heavy Rigid Truck Involvement',
#        'Articulated Truck Involvement', 'Speed Limit', 'Road User', 'Gender',
#        'Age', 'National Remoteness Areas', 'SA4 Name 2021',
#        'National LGA Name 2021', 'National Road Type', 'Christmas Period',
#        'Easter Period', 'Age Group', 'Day of week', 'Time of day']

In [ ]:
# List of columns to include
selected_columns = ['Crash ID', 'Year', 'Month', 'Day of week', 'Dayweek', 'Time', 'Christmas Period', 'Easter Period']

# Create a new DataFrame with the selected columns
date_df = filtered_df_2013_to_2023[selected_columns]

# Add a new "Date ID" column with values like 'date1', 'date2', 'date3', and so on
date_df['Date ID'] = ['date' + str(i) for i in range(1, len(date_df) + 1)]

# Reorder the columns with "Date ID" on the far left
date_df = date_df[['Date ID'] + ['Crash ID', 'Year', 'Month', 'Day of week', 'Dayweek', 'Time', 'Christmas Period', 'Easter Period']]

date_df


In [ ]:
# Save the filtered DataFrame to a new CSV file
date_df.to_csv('date_df.csv', index=False)


In [ ]:
# List of columns to include
selected_columns = ['Crash ID', 'Crash Type', 'Number of Fatalities', 'Road User', 'Gender', 'Age', 'Age Group']

# Create a new DataFrame with the selected columns
user_df = filtered_df_2013_to_2023[selected_columns]

# Add a new "Date ID" column with values like 'date1', 'date2', 'date3', and so on
user_df['User ID'] = ['date' + str(i) for i in range(1, len(date_df) + 1)]

# Reorder the columns with "Date ID" on the far left
user_df = date_df[['User ID'] + ['Crash ID', 'Crash Type', 'Number of Fatalities', 'Road User', 'Gender', 'Age', 'Age Group']]

user_df


In [ ]:
# Save the filtered DataFrame to a new CSV file
user_df.to_csv('user_df.csv', index=False)


In [ ]:
# List of columns to include
selected_columns = ['Crash ID', 'Bus Involvement', 'Heavy Rigid Truck Involvement', 'Articulated Truck Involvement']

# Create a new DataFrame with the selected columns
HGV_df = filtered_df_2013_to_2023[selected_columns]

# Add a new "Date ID" column with values like 'date1', 'date2', 'date3', and so on
HGV_df['HGV ID'] = ['date' + str(i) for i in range(1, len(date_df) + 1)]

# Reorder the columns with "Date ID" on the far left
HGV_df = date_df[['HGV ID'] + ['Crash ID', 'Bus Involvement', 'Heavy Rigid Truck Involvement', 'Articulated Truck Involvement']]

HGV_df


In [ ]:
# Save the filtered DataFrame to a new CSV file
HGV_df.to_csv('HGV_df.csv', index=False)


In [ ]:
# Create the Crash_df and add the "ID" columns to the Crash_df [ Date_ID, User_ID, HGV_ID]


# Data Analysis

In [ ]:
# Load the 'filtered_df' from the CSV file
filtered_df_2013_to_2023_df = pd.read_csv('filtered_data.csv')


In [ ]:
# the below analysis was performed on 'merged_df' so change merged_df to filtered_df

In [ ]:
# Create a histogram of ages
plt.figure(figsize=(8, 6))
sns.histplot(merged_df['Age'], bins=20)
plt.title('Age Distribution')
plt.xlabel('Age')
plt.ylabel('Count')
plt.show()

# NOTE # -9 represents a NaN and is creating the below '0' results, -9 will be cleaned out. 

In [ ]:
# What age group has the most likely car accidents each year?
age_group_yearly = merged_df.groupby(['Year', 'Age Group'])['Crash ID'].count().unstack()
age_group_yearly.plot(kind='bar', stacked=True)
plt.title('Age Group with Most Car Accidents Each Year')
plt.xlabel('Year')
plt.ylabel('Count')
plt.show()

# NOTE # -9 represents a NaN and will be cleaned out. 

In [ ]:
# Fatalities over years
fatalities_over_years = merged_df.groupby('Year')['Number of Fatalities'].sum()
fatalities_over_years.plot(kind='line')
plt.title('Fatalities Over the Years')
plt.xlabel('Year')
plt.ylabel('Count')
plt.show()


In [ ]:
# Fatalities per month
fatalities_per_month = merged_df.groupby('Month')['Number of Fatalities'].sum()
fatalities_per_month.plot(kind='bar')
plt.title('Fatalities per Month')
plt.xlabel('Month')
plt.ylabel('Count')
plt.show()



In [ ]:
# Map month numbers to month names
fatalities_per_month.index = fatalities_per_month.index.map(lambda x: calendar.month_name[x])

plt.figure(figsize=(10, 6))
fatalities_per_month.plot(kind='bar')
plt.title('Fatalities per Month')
plt.xlabel('Month')
plt.ylabel('Count')
plt.show()


In [ ]:
# Number of fatalities over the years for each state
# Create a line plot for fatality trends by state over the past decade
plt.figure(figsize=(12, 8))
sns.lineplot(data=merged_df, x='Year', y='Number of Fatalities', hue='State')
plt.title('Fatality Trends Over the Past Decade by State')
plt.xlabel('Year')
plt.ylabel('Number of Fatalities')
plt.legend(title='State')
plt.show()


In [ ]:
# Create a line plot that focuses only on the years after 2020.
plt.figure(figsize=(12, 8))
recent_years_df = merged_df[merged_df['Year'] >= 2020]
sns.lineplot(data=recent_years_df, x='Year', y='Number of Fatalities', hue='State')
plt.title('Fatality Trends After 2020 by State')
plt.xlabel('Year')
plt.ylabel('Number of Fatalities')
plt.legend(title='State')
plt.show()


In [ ]:
# Highest fatality per state
state_fatality = merged_df.groupby('State')['Number of Fatalities'].sum().sort_values(ascending=False)

plt.figure(figsize=(10, 6))
state_fatality.plot(kind='bar')
plt.title('Fatalities by State')
plt.xlabel('State')
plt.ylabel('Total Fatalities')
plt.show()


In [ ]:
# Create separate line plots for NT and Tas
plt.figure(figsize=(12, 8))
nt_df = merged_df[merged_df['State'] == 'NT']
sns.lineplot(data=nt_df, x='Year', y='Number of Fatalities', label='NT')
tas_df = merged_df[merged_df['State'] == 'Tas']
sns.lineplot(data=tas_df, x='Year', y='Number of Fatalities', label='Tas')
plt.title('Fatality Trends in NT and Tas Over the Years')
plt.xlabel('Year')
plt.ylabel('Number of Fatalities')
plt.legend(title='State')
plt.show()


In [ ]:
# Create a bar chart to show the number of crashes by road type
plt.figure(figsize=(10, 6))
sns.countplot(data=merged_df, y='National Road Type')
plt.title('Number of Crashes by Road Type')
plt.xlabel('Count')
plt.ylabel('Road Type')
plt.show()


In [ ]:
# Create a bar chart to show the distribution of accidents by time of day
plt.figure(figsize=(10, 6))
sns.countplot(data=merged_df, y='Time of Day')
plt.title('Accidents by Time of Day')
plt.xlabel('Count')
plt.ylabel('Time of Day')
plt.show()


In [ ]:
# Create a bar chart to show the distribution of accidents during Christmas and Easter periods
plt.figure(figsize=(8, 6))
sns.countplot(data=merged_df, x='Christmas Period')
plt.title('Accidents During Christmas Period')
plt.xlabel('Christmas Period')
plt.ylabel('Count')
plt.show()

plt.figure(figsize=(8, 6))
sns.countplot(data=merged_df, x='Easter Period')
plt.title('Accidents During Easter Period')
plt.xlabel('Easter Period')
plt.ylabel('Count')
plt.show()


In [ ]:
# Fatalities by Speed Limit
plt.figure(figsize=(10, 6))
sns.countplot(data=merged_df, x='Speed Limit', order=merged_df['Speed Limit'].value_counts().index)
plt.title('Fatalities by Speed Limit')
plt.xlabel('Speed Limit')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()
